# 🚀 Install, Import, and Log in

### Step 0️⃣: Install W&B

In [1]:
%%capture
#!pip install wandb
!pip install pandas

### Step 1️⃣: Import W&B and Login

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import os
import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split 

#%matplotlib inline

In [3]:
import wandb
from wandb.keras import WandbCallback

#wandb.init()  # defaults are over-ridden during the sweep


#wandb.login()

In [4]:

fluxData_df =pd.read_hdf("data/fluxData_df.h5", key="fluxData_df")

zernikeData_df =pd.read_hdf("data/zernikeData_df.h5", key="zernikeData_df")

Read data in

# 👩‍🍳 Prepare Dataset

In [5]:
# Prepare the training dataset
X = fluxData_df
y = zernikeData_df
x_train, x_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, random_state=42)


# Build simple Model

In [6]:
AoModel = keras.Sequential([
     keras.layers.InputLayer(19, name="digits"),
     keras.layers.Dense(2000, activation="relu"),
     keras.layers.Dense(1050, activation="relu"),
     keras.layers.Dense(100, activation="relu"),
     keras.layers.Dense(9, activation="linear", name="predictions"),

])
AoModel.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 2000)              40000     
_________________________________________________________________
dense_1 (Dense)              (None, 1050)              2101050   
_________________________________________________________________
dense_2 (Dense)              (None, 100)               105100    
_________________________________________________________________
predictions (Dense)          (None, 9)                 909       
Total params: 2,247,059
Trainable params: 2,247,059
Non-trainable params: 0
_________________________________________________________________


In [7]:
batch_size = 128
epochs = 200


    # optimizer='sgd',
    # loss='mse',
    # metrics=[tf.keras.metrics.RootMeanSquaredError()])



AoModel.compile(loss= keras.losses.MeanSquaredError(), optimizer="adam", metrics= [tf.keras.metrics.RootMeanSquaredError()])
#AoModel.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose = 2,  callbacks=[WandbCallback()])

#AoModel.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1, callbacks=[WandbCallback()])

# 🧠 Define the wandb sweep

In [8]:
os.environ["WANDB_NOTEBOOK_NAME"] = "AoKerasStudio.ipynb"

sweep_config = {
    "method": "random", # try grid or random
    "metric": {
        "name": "val_root_mean_squared_error",
        "goal": "minimize"
    },
    "parameters": {
        "booster": {
            "values": ["gbtree"]
        },
        "max_depth": {
      "min": 6,
      "max": 10
        },


    "learning_rate" :{
        "values": [0.04954]
        },        
    "subsample": {
        "values": [0.4372]
    },
    "colsample_bynode": {
        "values": [0.7597]
    },    
    "n_estimators": {
        "values": [3000, 10000, 20000]
    },
    "reg_alpha": {
        "values": [0.1668]
    },    
    "reg_lambda": {
        "values": [0.5222]
    }                
    }
}

In [9]:
## Define training loop

In [10]:
def train():
  config_defaults = {
  "booster": "gbtree",
  "max_depth": 3,
  "learning_rate": 0.1,
  "subsample": 1,
  "seed": 117,
  "test_size": 0.33,
}
  wandb.init(config=config_defaults)  # defaults are over-ridden during the sweep
  #config = wandb.config
  AoModel.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose = 2,validation_split = 0.2 , callbacks=[WandbCallback()])
  wandb.log({"error": "val_root_mean_squared_error"})

## Initialize sweep

In [11]:
sweep_id = wandb.sweep(sweep_config, entity = "uwe-sterr",project="KerasAoSweep")
wandb.run

Create sweep with ID: sy5zaaoc
Sweep URL: https://wandb.ai/uwe-sterr/KerasAoSweep/sweeps/sy5zaaoc


## Run sweep

In [12]:
wandb.agent(sweep_id, train, count=2)

wandb: Agent Starting Run: eew3xood with config:
wandb: 	booster: gbtree
wandb: 	colsample_bynode: 0.7597
wandb: 	learning_rate: 0.04954
wandb: 	max_depth: 9
wandb: 	n_estimators: 10000
wandb: 	reg_alpha: 0.1668
wandb: 	reg_lambda: 0.5222
wandb: 	subsample: 0.4372
wandb: Currently logged in as: uwe-sterr (use `wandb login --relogin` to force relogin)


Epoch 1/200
331/331 - 7s - loss: 0.0108 - root_mean_squared_error: 0.1039 - val_loss: 0.0048 - val_root_mean_squared_error: 0.0692
Epoch 2/200
331/331 - 7s - loss: 0.0039 - root_mean_squared_error: 0.0627 - val_loss: 0.0030 - val_root_mean_squared_error: 0.0548
Epoch 3/200
331/331 - 6s - loss: 0.0027 - root_mean_squared_error: 0.0523 - val_loss: 0.0022 - val_root_mean_squared_error: 0.0468
Epoch 4/200
331/331 - 6s - loss: 0.0021 - root_mean_squared_error: 0.0455 - val_loss: 0.0021 - val_root_mean_squared_error: 0.0456
Epoch 5/200
331/331 - 6s - loss: 0.0018 - root_mean_squared_error: 0.0420 - val_loss: 0.0015 - val_root_mean_squared_error: 0.0392
Epoch 6/200
331/331 - 6s - loss: 0.0015 - root_mean_squared_error: 0.0389 - val_loss: 0.0016 - val_root_mean_squared_error: 0.0395
Epoch 7/200
331/331 - 7s - loss: 0.0013 - root_mean_squared_error: 0.0362 - val_loss: 0.0021 - val_root_mean_squared_error: 0.0463
Epoch 8/200
331/331 - 7s - loss: 0.0012 - root_mean_squared_error: 0.0348 - val_los